# 📘 Conditional Workflow in LangGraph

## 🔍 What is a Conditional Workflow?

A **conditional workflow** in LangGraph is a dynamic execution path where the flow of nodes depends on certain **conditions** or **branching logic**. Instead of executing all nodes in a fixed order, LangGraph allows decision-making at runtime to determine which nodes should be triggered next, based on state values or outputs.

---

## 🧠 Key Concepts

### 1. **State**
- The shared memory (or context) passed between nodes.
- Conditions are evaluated based on the current state.

### 2. **Edges with Conditions**
- Each node can have multiple **edges** leading to different nodes.
- Edges can be conditionally activated using **functions** that return the name of the next node(s) based on the state.

### 3. **Conditional Edge Function**
- A Python function that takes `state` as input and returns:
  - A string (name of the next node)
  - Or a list of strings (multiple next nodes)

```python
def route(state):
    if state["feedback"] == "positive":
        return "follow_up"
    else:
        return "retry"
```

### ⚙️ When to Use Conditional Workflows

- ✅ Evaluating form responses (e.g., survey logic)
- ✅ Handling user inputs in chatbots (e.g., yes/no logic)
- ✅ Adaptive tutoring flows based on learner performance
- ✅ Early stopping or retries in LLM pipelines


### 📌 Structure Example (No Code)
- start node → evaluates prompt
- Conditional route based on sentiment:
- If sentiment = "positive" → go to thanks
- If sentiment = "negative" → go to retry
- Else → go to escalate

> This ensures that the workflow adapts its behavior based on real-time evaluations.



# 🎯 Problem Statement: Solving a Quadratic Equation using LangGraph Conditional Flow

## 🧩 Objective


Design a **LangGraph-based conditional workflow** to solve a quadratic equation of the form:

\[
ax^2 + bx + c = 0
\]

The workflow should intelligently **branch** based on the value of the **discriminant**:

\[
D = b^2 - 4ac
\]

---

## 📌 Goals

- Accept input values for `a`, `b`, and `c`.
- Calculate the **discriminant** `D`.
- Based on `D`, conditionally route to the correct solution path:
  - **Two Real Roots** if `D > 0`
  - **One Real Root** if `D == 0`
  - **No Real Roots** (complex roots) if `D < 0`
  
- Output the solution(s) accordingly.

---

## 🔁 Workflow Logic (Node Structure)

1. **Input Node**  
   - Collects values of `a`, `b`, and `c`.

2. **Discriminant Calculator Node**  
   - Computes `D = b² - 4ac`.

3. **Conditional Router Node**  
   - Based on `D`, routes to one of the following:
     - `TwoRootsNode` → if `D > 0` - 2 real notes
     - `OneRootNode` → if `D == 0` - 1 real root
     - `ComplexRootNode` → if `D < 0` - has no real roots

4. **Output Node**  
   - Returns the result in human-readable format.

---

## 🔍 Example Use Case

Given: `a = 1`, `b = -3`, `c = 2`  
Discriminant `D = (-3)^2 - 4(1)(2) = 1` → **Two distinct real roots**  
Workflow routes to `TwoRootsNode` and outputs the roots.

---

## ✅ Success Criteria

- Accurate root calculation based on input.
- Correct conditional routing using LangGraph.
- Clean, readable state transitions and outputs.

---


In [2]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Literal


In [3]:
class QuadState(TypedDict):
    a : int
    b : int
    c : int
    equation : str
    discriminant : float
    result : str

In [16]:
def show_equation(state : QuadState):
    equation = f'{state["a"]}x2 + {state["b"]}x {state["c"]}'
    return {'equation': equation}

def calculate_discriminant(state: QuadState):
    discriminant = state['b']**2 - (4*state['a']*state['c'])
    return {'discriminant': discriminant}


def real_roots(state:QuadState):
    root1 = (-state['b'] + (state['discriminant'])**0.5) /(2*state['a'])
    root2 = (-state['b'] - (state['discriminant'])**0.5) /(2*state['a'])

    result = f"The roots are {root1} and {root2}"
    return {'result':result}

def repeated_roots(state:QuadState):
    root = (-state['b']) /(2*state['a'])
    result = f"The ony repeated root is {root}"
    return {'result':result}

def no_real_roots(state:QuadState):
    result = "no real roots"
    return {'result':result}


In [20]:
## create condition

def check_condition(state:QuadState) -> Literal['real_roots', 'no_real_roots', 'repeated_roots']:
    if state['discriminant']>0:
        return "real_roots"
    elif state['discriminant']==0:
        return "repeated_roots"
    else:
        return "no_real_roots"

In [21]:
graph = StateGraph(QuadState)

## add nodes
graph.add_node("show_equation", show_equation)
graph.add_node("calculate_discriminant", calculate_discriminant)

graph.add_node("real_roots", real_roots)
graph.add_node("repeated_roots", repeated_roots)
graph.add_node("no_real_roots", no_real_roots)


## add edges
graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_discriminant')

graph.add_conditional_edges("calculate_discriminant",check_condition)
graph.add_edge('real_roots', END)
graph.add_edge('no_real_roots', END)
graph.add_edge('repeated_roots', END)




In [24]:
app = graph.compile()

In [30]:
initial_state = {'a':4, 'b':2, 'c':2}

app.invoke(initial_state)

{'a': 4,
 'b': 2,
 'c': 2,
 'equation': '4x2 + 2x 2',
 'discriminant': -28,
 'result': 'no real roots'}